# Experimental DECiSION code

In [16]:
RANDOM_STATE = 42
from numpy.random import seed
seed(RANDOM_STATE)

from tensorflow import set_random_seed
set_random_seed(RANDOM_STATE)

import random
random.seed = RANDOM_STATE

import DECiSION_settings as settings

from dltoolkit.utils.generic import model_architecture_to_file, model_summary_to_file, list_images
from dltoolkit.utils.image import normalise, normalise_single, standardise, standardise_single,\
    mean_subtraction
from dltoolkit.nn.segment import UNet_NN
from dltoolkit.utils.visual import plot_training_history
from dltoolkit.iomisc import HDF5Writer, HDF5Reader, HDF5Generator_Segment

from thesis_common import convert_img_to_pred, convert_pred_to_img,\
    group_images, show_image, read_images, read_groundtruths
from thesis_metric_loss import dice_coef, weighted_pixelwise_crossentropy_loss

from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

import numpy as np
import os, cv2, time, progressbar, shutil

import matplotlib.pyplot as plt
%matplotlib inline

## Test pre-processing

## Test reading batches of images

## Determine reduced image size

## Create augmented data

In [27]:
def create_random_patient_OLD(source_patient_ID, target_patient_ID,
                          source_path, target_path,
                          data_gen_args, settings, num_to_create=1, seed=42):
    """
    Create a new patient based on an existing one using data augmentation parameters
    :param patient_ID:
    :param data_aug:
    :return:
    """
    # Create the two identical data generators
    img_datagen = ImageDataGenerator(**data_gen_args)
    msk_datagen = ImageDataGenerator(**data_gen_args)

    # Construct the path to the source patient
    source_img_path = os.path.join(source_path, settings.FLDR_IMAGES + "/" + source_patient_ID)
    source_msk_path = os.path.join(source_path, settings.FLDR_GROUND_TRUTH + "/" + source_patient_ID)

    print(source_img_path)
    print(source_msk_path)
    
    # Construct the path to save the new patient to
    target_img_path = os.path.join(target_path, settings.FLDR_IMAGES + "/" + target_patient_ID)
    target_msk_path = os.path.join(target_path, settings.FLDR_GROUND_TRUTH + "/" + target_patient_ID)

    print(target_img_path)
    print(target_msk_path)
    
    # Create target folders, delete any existing folders
    if os.path.exists(target_img_path):
        shutil.rmtree(target_img_path)
    os.makedirs(target_img_path)

    if os.path.exists(target_msk_path):
        shutil.rmtree(target_msk_path)
    os.makedirs(target_msk_path)


    # Create the files
    for i in range(246):
        # Create the image and ground truth generators
        img_gen = img_datagen.flow_from_directory(img_path,
                                              target_size=(320, 320),
                                              color_mode='grayscale',
                                              class_mode=None,
                                              seed=seed,
                                              batch_size=1,
                                              save_to_dir=target_img_path,
                                              save_prefix=target_patient_ID,
                                              save_format="jpeg",
                                              shuffle=False,
                                             )
        next(img_gen)

        msk_gen = msk_datagen.flow_from_directory(msk_path,
                                              target_size=(320, 320),
                                              color_mode='grayscale',
                                              class_mode=None,
                                              seed=seed,
                                              batch_size=1,
                                              save_to_dir=target_msk_path,
                                              save_prefix=target_patient_ID,
                                              save_format="jpeg",
                                              shuffle=False,
                                             )

        next(msk_gen)

In [61]:
def create_random_patient(source_patient_ID, target_patient_ID,
                          source_path, target_path,
                          img_exts,
                          data_gen_args, settings, seed):
    """
    Create a new patient based on an existing one using data augmentation parameters
    :param patient_ID:
    :param data_aug:
    :return:
    """
    # Create the two identical data generators
    img_datagen = ImageDataGenerator(**data_gen_args)
    msk_datagen = ImageDataGenerator(**data_gen_args)

    # Construct the path to the source patient
    source_img_path = os.path.join(source_path, settings.FLDR_IMAGES + "/" + source_patient_ID)
    source_msk_path = os.path.join(source_path, settings.FLDR_GROUND_TRUTH + "/" + source_patient_ID)

    print("Source:")
    print(source_img_path)
    print(source_msk_path)
    
    # Construct the path to save the new patient to
    target_img_path = os.path.join(target_path, settings.FLDR_IMAGES + "/" + target_patient_ID)
    target_msk_path = os.path.join(target_path, settings.FLDR_GROUND_TRUTH + "/" + target_patient_ID)

    print("Target:")
    print(target_img_path)
    print(target_msk_path)
    
    # Create target folders, delete any existing folders
    if os.path.exists(target_img_path):
        shutil.rmtree(target_img_path)
    os.makedirs(target_img_path)

    if os.path.exists(target_msk_path):
        shutil.rmtree(target_msk_path)
    os.makedirs(target_msk_path)

    img_list = sorted(list(list_images(basePath=source_img_path, validExts=img_exts)))
    msk_list = sorted(list(list_images(basePath=source_msk_path, validExts=img_exts)))
    
    print("       # of images: {}".format(len(img_list)))
    print("# of ground truths: {}".format(len(msk_list)))

    # Create the files
    for i, (tmp_img, tmp_msk) in enumerate(zip(img_list, msk_list)):
        # Read the image
        img = cv2.imread(tmp_img, cv2.IMREAD_GRAYSCALE)
        img = np.reshape(img, (img.shape[0], img.shape[1], 1))

        # Create the generator
        img_gen = img_datagen.flow(np.asarray([img]),
                                              seed=seed,
                                              batch_size=1,
                                              save_to_dir=target_img_path,
                                              save_prefix=target_patient_ID + "_slice" + str(i+1) + "_",
                                              save_format="jpeg",
                                            )
        # Create the new image
        next(img_gen)

        # Read the ground truth
        msk = cv2.imread(tmp_msk, cv2.IMREAD_GRAYSCALE)
        msk = np.reshape(msk, (msk.shape[0], msk.shape[1], 1))

        msk_gen = msk_datagen.flow(np.asarray([msk]),
                                              seed=seed,
                                              batch_size=1,
                                              save_to_dir=target_msk_path,
                                              save_prefix=target_patient_ID + "_slice" + str(i+1) + "_",
                                              save_format="jpeg",
                                             )
        # Create the new ground truth
        next(msk_gen)

In [62]:
data_gen_args = dict(
    rotation_range=2.,
    width_shift_range=0.01,
    height_shift_range=0.01,
    shear_range=1.2,
    zoom_range=0.01,
    fill_mode='nearest'
)

src_patient_ID = "patient_1"
tgt_patient_ID = "patient_2"
tgt_patient_ID2 = "patient_3"
create_random_patient(patient_ID, tgt_patient_ID,
                      settings.TRAINING_PATH, settings.TRAINING_PATH,
                      ".jpg", data_gen_args, settings, seed=settings.RANDOM_STATE)

create_random_patient(patient_ID, tgt_patient_ID2,
                      settings.TRAINING_PATH, settings.TRAINING_PATH,
                      ".jpg", data_gen_args, settings, seed=settings.RANDOM_STATE * 3)



Source:
../data/MSC8002/training/images/patient_1
../data/MSC8002/training/groundtruths/patient_1
Target:
../data/MSC8002/training/images/patient_2
../data/MSC8002/training/groundtruths/patient_2
       # of images: 247
# of ground truths: 247
Source:
../data/MSC8002/training/images/patient_1
../data/MSC8002/training/groundtruths/patient_1
Target:
../data/MSC8002/training/images/patient_3
../data/MSC8002/training/groundtruths/patient_3
       # of images: 247
# of ground truths: 247


In [9]:
import os

data_gen_args = dict(
#     rotation_range=2.,
#     width_shift_range=0.01,
#     height_shift_range=0.01,
#     shear_range=1.2,
    zoom_range=0.01,
#     fill_mode='nearest'
)

# create_random_patient()

img_datagen = ImageDataGenerator(**data_gen_args)
msk_datagen = ImageDataGenerator(**data_gen_args)

seed = 1
PATIENT_ID = "patient_1"
# img_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_IMAGES + "/" + PATIENT_ID)
# msk_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_GROUND_TRUTH + "/" + PATIENT_ID)

# img_path = os.path.join("../data/1", PATIENT_ID)
# msk_path = os.path.join("../data/1", PATIENT_ID)

img_path = os.path.join("../data/1/images", "")
msk_path = os.path.join("../data/1/groundtruths", "")


print(img_path)
img_gen = img_datagen.flow_from_directory(img_path,
                                          target_size=(320, 320),
                                          color_mode='grayscale',
                                          class_mode=None,
                                          seed=seed,
                                          batch_size=246,
                                          save_to_dir="/Users/geoff/Documents/Development/DLToolkit/data/1/tmp/images",
                                          save_prefix="images_",
                                          save_format="jpeg",
                                          shuffle=False,
                                         )



msk_gen = msk_datagen.flow_from_directory(msk_path,
                                          target_size=(320, 320),
                                          color_mode='grayscale',
                                          class_mode=None,
                                          seed=seed,
                                          batch_size=246,
                                          save_to_dir="/Users/geoff/Documents/Development/DLToolkit/data/1/tmp/groundtruths",
                                          save_prefix=targ"groundtruths_",
                                          save_format="jpeg",
                                          shuffle=False,
                                         )

lala = next(img_gen)
hoho = next(msk_gen)

print(lala.shape, lala.dtype)
print(hoho.shape, hoho.dtype)

../data/1/images/
Found 246 images belonging to 1 classes.
Found 246 images belonging to 1 classes.
(246, 320, 320, 1) float32
(246, 320, 320, 1) float32


In [ ]:
img = cv2.imread("../data/1/groundtruths/patient_1/S57208Filter0236.jpg", cv2.IMREAD_GRAYSCALE)
print(img.shape, img.dtype, np.sum(img))

img = cv2.imread("../data/1/tmp/groundtruths/groundtruths__236_9828491.jpeg", cv2.IMREAD_GRAYSCALE)
print(img.shape, img.dtype, np.sum(img))

# Plan B

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing.image import random_rotation
img_exts = ".jpg"

seed = 1
PATIENT_ID = "patient_1"
img_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_IMAGES + "/" + PATIENT_ID)
msk_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_GROUND_TRUTH + "/" + PATIENT_ID)

img_files = sorted(list(list_images(basePath=img_path, validExts=img_exts)))
msk_files = sorted(list(list_images(basePath=msk_path, validExts=img_exts)))

for i, (img, msk) in enumerate(zip(img_files, msk_files)):
#     print(img)
#     print(msk,"\n")
    
    tmp_img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    tmp_img = np.reshape(tmp_img, (tmp_img.shape[0], tmp_img.shape[1], 1))

    tmp_msk = cv2.imread(msk, cv2.IMREAD_GRAYSCALE)
    tmp_msk = np.reshape(tmp_msk, (tmp_msk.shape[0], tmp_msk.shape[1], 1))

    if i == 236:
        print(img)
        print(msk)

#         tmp_img = random_rotation(tmp_img, 2, row_axis=0, col_axis=2, channel_axis=2)
        show_image(np.squeeze(tmp_img), "img")

#         tmp_msk = random_rotation(tmp_msk, 2, row_axis=0, col_axis=2, channel_axis=2)
        show_image(np.squeeze(tmp_msk), "msk")


